# Web-scraping: сбор данных из баз данных и интернет-источников

*Алла Тамбовцева*

## Практикум 3. Формат JSON и его обработка в рамках блока кода JavaScript

Импортируем все необходимое:

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

### Словари и датафреймы

Если нам нужно хранить много информации разного уровня в Python, это удобно делать с помощью словарей (тип `dict`):

In [2]:
info = {"database" : "HP", 
 "data" : [{"name" : "Dobby", "type" : "house-elf"}, 
           {"name": "Harry Potter", "type": "human"}, 
           {"name" : "Griphook", "type" : "goblin"}]}
info

{'database': 'HP',
 'data': [{'name': 'Dobby', 'type': 'house-elf'},
  {'name': 'Harry Potter', 'type': 'human'},
  {'name': 'Griphook', 'type': 'goblin'}]}

В таком же виде удобно хранить информацию где-то на сервере, чтобы в нужный момент выбирать по ключу подходящие данные и подставлять в нужное место на страницу сайта. Так, в примере выше, известно, что есть база данных `HP`, в ней есть набор данных `data`, в котором есть характеристики разных героев из вселенной Дж.Роулинг. 

Как преобразовать данные выше в более привычный табличный формат? Вспомнить, что функция `DataFrame()` из `pandas` умеет преобразовывать списки словарей в датафреймы (один словарь внутри списка равен одной строке таблицы). Давайте выберем список с характеристиками героев и получим удобную таблицу!

In [3]:
pd.DataFrame(info["data"])

,name,type
0,Dobby,house-elf
1,Harry Potter,human
2,Griphook,goblin


Теперь давайте разберемся, в каком виде могут храниться подобного вида данные в исходном коде веб-страницы.

### Формат JSON

JSON (от *JavaScript Object Notation*) – текстовый формат хранения данных, изначально использовался в языке JavaScript, но затем стал универсальным машиночитаемым форматом, распознаваемым разными языками программирования. Различают:

* JSON-строки (текст с определённой структурой данных внутри);
* JSON-файлы (текстовые файлы с расширением `.json` со строкой JSON-внутри).

Какие структуры данных Python могут встретиться внутри JSON-строки? Знакомые нам списки и словари!

**Пример JSON-строки, содержащей списки:**

In [4]:
# фрагмент результатов голосования в Арбитражный комитет Википедии: 
# время голосования, голос, кандидат, избиратель:

example01 = """
[["2008-11-23 00:32:00", "-", "Solon", "Kalan"], 
  ["2008-11-23 00:34:00", "+", "Chronicler", "Altes"], 
  ["2008-11-23 00:34:00", "+", "Ilya Voyager", "Altes"]]
"""

**Пример JSON-строки, содержащей словари:**

In [5]:
# фрагмент результатов голосования за актеров 
# на сайте kinoteatr.ru

example02 = """
[{ "id":"16804", "plus":"131", "minus":"4", "voted":"" },
{ "id":"56008", "plus":"91", "minus":"10", "voted":"" },
{ "id":"62460", "plus":"94", "minus":"4", "voted":"" }]
"""

Этот формат хранения данных удобен своей универсальностью. Во-первых, он позволяет сохранять и выгружать в компактные текстовые файлы данные со сложной структурой (например, словари, внутри которых есть ещё словари). Во-вторых, формат JSON не привязан к какому-то конкретному языку программирования. Можно создать список списков в Python, выгрузить его в строку JSON, затем считать эту строку с помощью другого языка и получить результат в виде аналогичных структур данных, принятых в этом языке (например, аналогом питоновского словаря `dict` в языке R может выступить поименованный вектор или фрейм `list`).

По этим причинам формат JSON очень популярен. Его можно встретить при работе с географическими данными (файлы с особым расширением `.geojson`, которые содержат метки с координатами объектов), при парсинге HTML-страниц (файлы `.json`, из которых «подтягивается» регулярно обновляемая информация для построения всяких интерактивных визуализаций на сайте) и при подключении к API – интерфейсам, которые можно использовать как базы данных для автоматизированной выгрузки данных из приложений и социальных сетей.

В этом практикуме мы поработаем с JSON-строками. Импортируем необходимый для работы модуль `json`. Этот модуль `json` – базовый (как знакомые нам `requests` или `os`), он не требует дополнительной установки.  

In [6]:
import json

В модуле `json` есть две функции, `loads()` и `load()`. Первая преобразует данные из обычной строки (как здесь), вторая – данные, загруженные из файла с расширением `.json`. Преобразование JSON-строки в структуру данных называется **десериализацией JSON**. 

Обратная операция – превращение структуры данных в Python в JSON-строку – тоже существует, и называется она **сериализацией JSON**. Для сериализации используется аналогичная пара функций, `dumps()` и `dump()`. Первая будет превращать структуру данных в JSON-строку, вторая – превращать структуру данных в строку и выгружать эту строку в файл с расширением `.json`.

В качестве наглядного примера десериализуем строки `example01` и `example02` – превратим валидные (корректные, где все скобки и кавычки на месте) JSON-сроки в питоновские списки и словари:

In [7]:
# строка с текстом -> список списков
json.loads(example01)

[['2008-11-23 00:32:00', '-', 'Solon', 'Kalan'],
 ['2008-11-23 00:34:00', '+', 'Chronicler', 'Altes'],
 ['2008-11-23 00:34:00', '+', 'Ilya Voyager', 'Altes']]

In [8]:
# строка с текстом -> список словарей
json.loads(example02)

[{'id': '16804', 'plus': '131', 'minus': '4', 'voted': ''},
 {'id': '56008', 'plus': '91', 'minus': '10', 'voted': ''},
 {'id': '62460', 'plus': '94', 'minus': '4', 'voted': ''}]

И сериализуем словарь `info`:

In [9]:
# словарь -> текст
json.dumps(info)

'{"database": "HP", "data": [{"name": "Dobby", "type": "house-elf"}, {"name": "Harry Potter", "type": "human"}, {"name": "Griphook", "type": "goblin"}]}'

In [10]:
# словарь -> файл .json
f = open("info.json", "w")
json.dump(info, f)
f.close()

### Извлечение кода JavaScript из HTML и обработка JSON-строк

Вернемся к задаче сбора ссылок на новости по дате. В предыдущем практикуме мы написали код, который находит нужный фрагмент с кодом на JavaScript (с тэгом `<script>`), в котором явно есть ссылки на новости:

In [11]:
url = "https://nplus1.ru/news/2025/01/03"
soup = BeautifulSoup(requests.get(url).text)
js = soup.find_all("script")[8].text

# print(js)

**NB.** Если после исполнения кода выше в `js` пустая строка с текстом – замените `.text` на `.next`, это особенность другой версии `bs4`:

    soup.find_all("script")[8].next
    
Так как формально внутри `<script>` не текст, а код JavaScript, текст через атрибут `.text` в более новой версии `bs4` не находится (более строгое отношение к типам). А атрибут `.next` возвращает содержимое, следующее за тэгом, что в нашем случае – ровно то, что нужно.

Объект `js` сейчас – это обычная строка (текст) с кодом на языке JavaScript. Знать этот язык нам совсем не обязательно, достаточно понять, какой фрагмент текста содержит ссылки. В данном случае текст в `js` выглядит жутко, это из-за того, что двойные кавычки местами представлены в виде символов Unicode. Выполним замену, чтобы меньше пугаться:

In [12]:
js = js.replace("\\u0022", '"')

Чтобы еще меньше пугаться, давайте заменим блоки из `\` на один слэш:

In [13]:
js_clean = js.replace("\\\\\\", "")

В целом, уже сейчас можно попытаться найти в `js_clean` фрагменты со ссылками на новости, но это неудобно. Поэтому давайте выберем текст после `JSON.parse(` и попробуем привести его к «чистой» JSON-строке!

In [14]:
# разбиваем по JSON.parse(\' и забираем все, что после (индекс 1)
# метод .strip() без текста внутри убирает отступы в начале/конце строки,
# а с текстом внутри – убирает этот текст в начале/конце строки

final = js_clean.split("JSON.parse(\'")[1].strip().strip("\');")
# final

Теперь эту строку можно десериализовать – превратить в словарь!

In [15]:
D = json.loads(final)

Давайте выберем из словаря запись с нужными данными и превратим их в таблицу!

In [16]:
df = pd.DataFrame(D["data"])
df

,type,title,typeKey,isImage,difficulty,themes,url,imageBg,rubrics,isVisibleTime,author,date,image,time,titleClear,subtitle,imageBox
0,default,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,news,True,"{'link': '/material/difficulty/2', 'value': 2....",[],https://nplus1.ru/news/2025/01/03/shocked-iron...,#ffffff,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...",False,"{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...",None,"{'link': '/search/empty/', 'value': '14:15', '...",None,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'original': {'hex': None, 'src': 'https://min..."
1,default,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,news,True,"{'link': '/material/difficulty/3', 'value': 3....",[],https://nplus1.ru/news/2025/01/03/mesolithic-p...,#ffffff,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...",False,"{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...",None,"{'link': '/search/empty/', 'value': '11:25', '...",None,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'original': {'hex': None, 'src': 'https://min..."
2,default,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,news,True,"{'link': '/material/difficulty/2', 'value': 2....",[],https://nplus1.ru/news/2025/01/03/knitted-fabr...,#ffffff,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...",False,"{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...",None,"{'link': '/search/empty/', 'value': '09:20', '...",None,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'original': {'hex': None, 'src': 'https://min..."


### Задача 1

Выберите из полученного датафрейма столбец со ссылками и превратите его в массив или список ссылок (тип `list` или `array`).

In [17]:
# объект Pandas Series, 
# пары «номер строки – значение»

df["url"]

0    https://nplus1.ru/news/2025/01/03/shocked-iron...
1    https://nplus1.ru/news/2025/01/03/mesolithic-p...
2    https://nplus1.ru/news/2025/01/03/knitted-fabr...
Name: url, dtype: object

In [18]:
# объект Numpy Array, массив,
# перечень значений без номеров

df["url"].values

array(['https://nplus1.ru/news/2025/01/03/shocked-iron-melting',
       'https://nplus1.ru/news/2025/01/03/mesolithic-people-from-karelia',
       'https://nplus1.ru/news/2025/01/03/knitted-fabric-shapes'],
      dtype=object)

In [19]:
# обычный список

L = list(df["url"].values)
L

['https://nplus1.ru/news/2025/01/03/shocked-iron-melting',
 'https://nplus1.ru/news/2025/01/03/mesolithic-people-from-karelia',
 'https://nplus1.ru/news/2025/01/03/knitted-fabric-shapes']

### Задача 2

Напишите функцию `get_url_by_date()`, которая принимает на вход ссылку на страницу новостей за определенную дату и возвращает список ссылок на все новости в этот день.

In [20]:
def get_url_by_date(url):
    soup = BeautifulSoup(requests.get(url).text)
    js = soup.find_all("script")[8].text
    js = js.replace("\\u0022", '"')
    js_clean = js.replace("\\\\\\", "")
    final = js_clean.split("JSON.parse(\'")[1].strip().strip("\');")
    D = json.loads(final)
    df = pd.DataFrame(D["data"])
    L = list(df["url"].values)
    return L

In [21]:
# проверяем
get_url_by_date("https://nplus1.ru/news/2025/01/08/")

['https://nplus1.ru/news/2025/01/08/two-dimensional-salt',
 'https://nplus1.ru/news/2025/01/08/lilium-episode-four-new-hope',
 'https://nplus1.ru/news/2025/01/08/choose-the-right-regimen',
 'https://nplus1.ru/news/2025/01/08/meningocele-cured',
 'https://nplus1.ru/news/2025/01/08/watch-the-larynx',
 'https://nplus1.ru/news/2025/01/08/doxy-pep-ok']

**NB.** Так как есть дни, в которые новости не публиковались, функция выше не универсальна. В случае отсутствия новостей на странице она будет выдавать ошибку при запуске строки `L = list(df["url"].values)`, так как внутри `df` – пустая таблица, а значит, столбца с названием `url` там нет (`KeyError`). 

Исправим это – добавим развилку с `try-except`:

In [22]:
def get_url_by_date(url):
    soup = BeautifulSoup(requests.get(url).text)
    js = soup.find_all("script")[8].text
    js = js.replace("\\u0022", '"')
    js_clean = js.replace("\\\\\\", "")
    final = js_clean.split("JSON.parse(\'")[1].strip().strip("\');")
    D = json.loads(final)
    df = pd.DataFrame(D["data"])
    try:
        L = list(df["url"].values)
    except:
        L = []
    return L

In [23]:
# проверяем – 1-го января новостей нет
get_url_by_date("https://nplus1.ru/news/2025/01/01/")

[]

Теперь мы можем совместить код из практикума 2.2 для получения ссылок на страницы всех дат (задача №2 в сюжете 2) и код выше, чтобы получить программу, которая сформирует список ссылок на все новости за 2024 год.

Однако мы можем пойти дальше и получить гораздо больше информации из датафрейма, который получился преобразованием JSON-строки с кода внутри `<script>` на странице. Давайте в свободном режиме изучим структуру этого датафрейма!

In [24]:
# полезный код для перекодирования текста

"\\u0422\\u0435".encode().decode('unicode_escape')

'Те'

Давайте оставим в `df` только те столбцы, из которых можно забрать полезную информацию:

* `title`: заголовок;
* `subtitle`: подзаголовок;
* `difficulty`: сложность новости;
* `url`: ссылка на новость;
* `rubrics`: рубрики новости;
* `author`: автор;
* `date` и `time`: дата и время публикации.

In [25]:
small = df[["title", "subtitle", "difficulty", "url", 
            "rubrics", "author", "date", "time"]]
small

,title,subtitle,difficulty,url,rubrics,author,date,time
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '..."
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '..."
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '..."


In [26]:
# выберем столбец с заголовком новости title
# применим ко всем ячейкам lambda-функцию, которая
# перекодирует текст в понятный вид

small["title_c"] = small["title"].apply(lambda x: x.encode().decode('unicode_escape'))
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани


Перекодируем подзаголовок новости по аналогии с заголовком:

In [27]:
small["subtitle_c"] = small["subtitle"].apply(lambda x: x.encode().decode('unicode_escape'))
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c,subtitle_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным


Заберем из столбца со сложностью только числовое значение сложности. Внутри одной ячейки в столбце `difficulty` – словарь, а значит, из словаря можно забрать запись по подходящему ключу (здесь `value`):

In [28]:
small["difficulty"][0] # одна ячейка

{'link': '/material/difficulty/2', 'value': 2.9, 'type': 'difficulty'}

In [29]:
# применяем функцию, которая вытащит value,
# обновляем столбец difficulty

small["difficulty_c"] = small["difficulty"].apply(lambda x: x.get("value"))
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c,subtitle_c,difficulty_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина,2.9
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад,3.1
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным,2.3


Проделаем то же с датой и временем:

In [30]:
small["date_c"] = small["date"].apply(lambda x: x.get("value"))
small["time_c"] = small["time"].apply(lambda x: x.get("value"))
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c,subtitle_c,difficulty_c,date_c,time_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина,2.9,03.01.25,14:15
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад,3.1,03.01.25,11:25
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным,2.3,03.01.25,09:20


Теперь извлечем имя автора – только его нужно еще перекодировать:

In [31]:
small["author_c"] = small["author"].apply(lambda x: 
                                          x.get("name").encode().decode('unicode_escape'))
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c,subtitle_c,difficulty_c,date_c,time_c,author_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина,2.9,03.01.25,14:15,Егор Конюхов
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад,3.1,03.01.25,11:25,Михаил Подрезов
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным,2.3,03.01.25,09:20,Егор Конюхов


Проделаем то же с рубриками. Однако здесь стоит учесть, что рубрик может быть несколько, поэтому в одной ячейке столбца не один словарь, а список словарей:

In [32]:
# даже если рубрика одна, все равно список – для универсальности
small["rubrics"][0]

[{'id': 837,
  'name': '\\u0424\\u0438\\u0437\\u0438\\u043a\\u0430',
  'link': 'https://nplus1.ru/search?tags=869',
  'type': 'rubric'}]

Напишем отдельную функцию, которая извлекает из такого списка все словари, а из каждого словаря – рубрику:

In [33]:
# rubrics – список из рубрик в непонятном виде
# joined – одна строка с рубриками через запятую с пробелом
# decoded – одна строка с рубриками через запятую с пробелом, уже в понятном виде

def get_rubs(rubs_list):
    rubrics = [r.get("name") for r in rubs_list]
    joined = ", ".join(rubrics)
    decoded = joined.encode().decode('unicode_escape')
    return decoded

In [34]:
small["rubrics_c"] = small["rubrics"].apply(get_rubs)
small # см столбцы в конце

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,title,subtitle,difficulty,url,rubrics,author,date,time,title_c,subtitle_c,difficulty_c,date_c,time_c,author_c,rubrics_c
0,\u0422\u0435\u043c\u043f\u0435\u0440\u0430\u04...,\u041d\u043e\u0432\u0430\u044f \u043e\u0446\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/shocked-iron...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '14:15', '...",Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина,2.9,03.01.25,14:15,Егор Конюхов,Физика
1,\u0411\u043e\u043b\u044c\u0448\u0438\u043d\u04...,\u041e\u043d\u0438 \u0436\u0438\u043b\u0438 \u...,"{'link': '/material/difficulty/3', 'value': 3....",https://nplus1.ru/news/2025/01/03/mesolithic-p...,"[{'id': 844, 'name': '\u0410\u0440\u0445\u0435...","{'id': 275, 'name': '\u041c\u0438\u0445\u0430\...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '11:25', '...",Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад,3.1,03.01.25,11:25,Михаил Подрезов,Археология
2,\u0424\u0438\u0437\u0438\u043a\u0438 \u0440\u0...,\u041e\u043d\u043e \u043e\u043a\u0430\u0437\u0...,"{'link': '/material/difficulty/2', 'value': 2....",https://nplus1.ru/news/2025/01/03/knitted-fabr...,"[{'id': 837, 'name': '\u0424\u0438\u0437\u0438...","{'id': 470, 'name': '\u0415\u0433\u043e\u0440 ...","{'link': '/news/2025/01/03', 'value': '03.01.2...","{'link': '/search/empty/', 'value': '09:20', '...",Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным,2.3,03.01.25,09:20,Егор Конюхов,Физика


Всё! Можем выбрать только новые столбцы с «чистой» информацией:

In [35]:
tochno_final = small[["title_c", "subtitle_c", "difficulty_c", 
                      "date_c", "time_c", "author_c", "rubrics_c"]]
tochno_final

,title_c,subtitle_c,difficulty_c,date_c,time_c,author_c,rubrics_c
0,Температуру ядра Земли уточнили с помощью лазе...,Новая оценка составила 6202 кельвина,2.9,03.01.25,14:15,Егор Конюхов,Физика
1,Большинство людей эпохи мезолита из Оленеостро...,Они жили около восьми тысяч лет назад,3.1,03.01.25,11:25,Михаил Подрезов,Археология
2,Физики разобрались в состоянии покоя ткани,Оно оказалось не единственным,2.3,03.01.25,09:20,Егор Конюхов,Физика


В целом, если нам сам текст новостей не очень нужен, мы можем написать функцию, которая по ссылке на конкретную дату возвращает таблицу с полным описанием всех новостей :)

P.S. На предупреждение на красном фоне можно не обращать внимание, pandas сообщает нам, что в более новых версиях библиотеки при использовании `.apply()` для создания новых столбцов на основе старых лучше выбирать столбцы более сложным образом. Но пока все работает, предлагаю просто выключить это предупреждение – можете добавить эту строчку в начало файла после импорта `pandas`:

    pd.set_option('chained_assignment', None)